# Variable dependiente: Personas Corrupción Amplia (numérica)

## 1. Data

#### 1.1. Import libraries and data

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as  pd, numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [3]:
import funciones as fun
import variables_nombres as vn

In [4]:
path = r'..\..\output\data_preprocess\dfs_0_i_pca.csv'
data = pd.read_csv( path )

In [5]:
data

,sexo,facebook,web,mue_pro_1,mue_pro_4,mue_pro_5,mue_pro_6,mue_pro_7,mue_pro_10,mue_pro_13,...,piagtotfun_f5viv,tejgtotfun_f5viv,tdvgtotfun_f5viv,devppimtotfun_f5viv,tipo_control,corrup_intensa,corrup_amplia,per_corrup1,per_corrup2,monto_corrup2
0,1.0,0.0,1.0,1.0,2.0,0.0,0.0,2.0,2.0,2.0,...,0.000000,0.000000,0.000000,0.000000,2.0,1.0,1.0,1.791759,0.000000,15.851884
1,1.0,0.0,0.0,1.0,1.0,1.0,0.0,2.0,2.0,2.0,...,0.000000,9.648660,9.648660,4.230487,2.0,1.0,1.0,1.386294,0.000000,12.060607
2,2.0,0.0,0.0,1.0,2.0,0.0,0.0,2.0,2.0,1.0,...,0.000000,9.159152,9.159152,4.615121,1.0,1.0,1.0,1.609438,0.000000,15.537098
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,1.0,1.0,1.0,2.079442,0.000000,15.031823
4,2.0,1.0,0.0,1.0,1.0,2.0,0.0,1.0,0.0,0.0,...,0.000000,0.000000,12.583653,3.811629,2.0,1.0,1.0,2.197225,1.609438,12.206308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692,2.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,...,13.759167,12.402012,12.402012,4.615106,2.0,0.0,1.0,0.000000,1.098612,19.670970
693,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,2.0,0.0,1.0,0.000000,2.197225,13.223206
694,2.0,1.0,1.0,1.0,1.0,5.0,0.0,1.0,1.0,1.0,...,12.105517,12.350079,12.350079,4.613931,2.0,0.0,0.0,0.000000,0.000000,0.000000
695,2.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,...,0.000000,14.455608,14.455183,4.323754,2.0,1.0,1.0,2.833213,0.000000,16.768403


In [6]:
# Borrar otras variables dependientes. Solo nos quedamos con 'monto_corrup2'
data = data.drop( [ 'tipo_control', 'corrup_intensa', 
                    'corrup_amplia', 'per_corrup1', 
                    'monto_corrup2' ], 
                  axis = 1 )

In [7]:
data = data.dropna( axis = 1 )

#### 1.2. Split data into training and test set

In [8]:
np.random.seed( 1234 )
training = np.random.choice( data.index, size=int( len( data )*( 3/4 )), replace = False )

data_train = data.loc[ training, : ]
data_test = data.drop( training, axis = 0 )

## 2. Definir el modelo

In [9]:
import statsmodels.formula.api as smf
import patsy

In [10]:
# varnames =  data.columns.values.tolist()
# varnames = pd.DataFrame( varnames )
# varnames.to_excel( r'..\..\extra\vars_list\pca.xlsx' )

#### 2.1. Construir modelos

In [11]:
%%time

formula_basic = "per_corrup2 ~ tejgfun_ct05pgercon +  dfgpimpiafun_ct05pgercon +  devppimfun_ct05pgercon +  dfgpimpiafun_ct05opseg +  devppimfun_ct05opseg +  devppimfun_ct05trab +  piagfun_ct05come +  dfgdevpiagfun_ct05turi +  devppimfun_ct05turi +  tejgfun_ct05agro +  tdvgfun_ct05agro +  devppimfun_ct05agro +  piagfun_ct05energia +  tejgfun_ct05energia +  tdvgfun_ct05energia +  dfgpimpiafun_ct05ind +  tejgfun_ct05trans +  dfgpimpiafun_ct05trans +  devppimfun_ct05trans +  dfgdevpiagfun_ct05amb +  devppimfun_ct05amb +  pimgfun_ct05san +  tejgfun_ct05san +  dfgpimpiafun_ct05viv +  devppimfun_ct05viv +  tejgfun_ct05protsoc +  dfgdevpiagfun_ct05protsoc +  devppimfun_ct05protsoc +  tejgfun_ct05prevsoc +  dfgdevpiagfun_ct05prevsoc +  devppimfun_ct05dpub +  tejgfun_ct06pgercon +  devppimfun_ct06pgercon +  piagfun_ct06opseg +  tejgfun_ct06opseg +  dfgpimpiafun_ct06opseg +  devppimfun_ct06opseg +  pimgfun_ct06trab +  devppimfun_ct06trab +  tejgfun_ct06come +  dfgdevpiagfun_ct06come +  pimgfun_ct06turi +  tdvgfun_ct06turi +  dfgpimpiafun_ct06turi +  dfgdevpiagfun_ct06turi +  devppimfun_ct06turi +  dfgpimpiafun_ct06pesca +  tejgfun_ct06ind +  dfgpimpiafun_ct06trans +  dfgdevpiagfun_ct06trans +  devppimfun_ct06trans +  pimgfun_ct06amb +  piagfun_ct06san +  tdvgfun_ct06san +  devppimfun_ct06san +  dfgdevpiagfun_ct06viv +  devppimfun_ct06viv +  devppimfun_ct06salud +  dfgpimpiafun_ct06edu +  tejgfun_ct06protsoc +  dfgpimpiafun_ct06protsoc +  devppimfun_ct06protsoc +  dfgdevpiagft_rdet +  devppimft_rdet +  piagrb_reod +  tejgrb_reod +  devppimrb_reod +  dfgpimpiarb_redr +  dfgdevpiagrb_redr +  piagrb_dotr +  tejgrb_dotr +  tejgrb_fonc +  dfgdevpiagrb_fonc +  devppimrb_fonc +  devppimrb_canr +  devppimct_r00gstcr +  tejgct_r13gstcr +  tdvgct_r13gstcr +  tejgct_r07srdeu +  devppimct_r07srdeu +  pimgge_r00ct05pobso +  devppimge_r00ct05pobso +  devppimge_r00ct05popso +  tejgge_r00ct05biser +  devppimge_r00ct05biser +  pimgge_r00ct05otgst +  devppimgge_r00ct05otgst +  dfgdevpiagge_r00ct06dotra +  devppimgge_r00ct06otgst +  tdvgge_r00ct06acanf +  tejgge_r09ct05pobso +  tdvgge_r09ct05pobso +  dfgpimpiagge_r09ct05pobso +  dfgdevpiagge_r09ct05pobso +  devppimgge_r09ct05pobso +  tejgge_r09ct05popso +  dfgdevpiagge_r09ct05popso +  devppimgge_r09ct05popso +  devppimgge_r09ct05biser +  pimgge_r09ct05dotra +  tejgge_r09ct05dotra +  dfgdevpiagge_r09ct05dotra +  devppimgge_r09ct05dotra +  piagge_r09ct05otgst +  tdvgge_r09ct05otgst +  dfgdevpiagge_r09ct05otgst +  devppimgge_r09ct05otgst +  piagge_r09ct06dotra +  pimgge_r09ct06dotra +  devppimgge_r09ct06dotra +  piagge_r09ct06otgst +  tejgge_r09ct06otgst +  dfgdevpiagge_r09ct06otgst +  devppimgge_r09ct06otgst +  tejgge_r09ct06acanf +  devppimgge_r09ct06acanf +  dfgpimpiagge_r09ct07sedpu +  dfgdevpiagge_r09ct07sedpu +  devppimgge_r19ct05biser +  dfgpimpiagge_r19ct06dotra +  dfgdevpiagge_r19ct06dotra +  devppimgge_r19ct06dotra +  tejgge_r19ct06acanf +  dfgdevpiagge_r13ct05pobso +  tejgge_r13ct05popso +  dfgpimpiagge_r13ct05popso +  devppimgge_r13ct05popso +  devppimgge_r13ct05biser +  piagge_r13ct05dotra +  tejgge_r13ct05dotra +  devppimgge_r13ct05dotra +  dfgpimpiagge_r13ct05otgst +  dfgdevpiagge_r13ct05otgst +  devppimgge_r13ct05otgst +  dfgpimpiagge_r13ct06dotra +  devppimgge_r13ct06dotra +  pimgge_r07ct05pobso +  tejgge_r07ct05pobso +  dfgpimpiagge_r07ct05pobso +  dfgdevpiagge_r07ct05pobso +  devppimgge_r07ct05pobso +  piagge_r07ct05popso +  dfgdevpiagge_r07ct05popso +  devppimgge_r07ct05popso +  tdvgge_r07ct05biser +  dfgdevpiagge_r07ct05biser +  devppimgge_r07ct05biser +  tejgge_r07ct05dotra +  tdvgge_r07ct05dotra +  dfgpimpiagge_r07ct05dotra +  devppimgge_r07ct05dotra +  pimgge_r07ct05otgst +  tejgge_r07ct05otgst +  dfgdevpiagge_r07ct05otgst +  devppimgge_r07ct05otgst +  dfgdevpiagge_r08ct05pobso +  devppimge_r08ct05pobso +  piagge_r08ct05popso +  dfgdevpiagge_r08ct05popso +  devppimge_r08ct05popso +  dfgpimpiage_r08ct05biser +  dfgdevpiagge_r08ct05biser +  tejgge_r08ct05dotra +  tdvgge_r08ct05dotra +  dfgpimpiage_r08ct05dotra +  devppimge_r08ct05dotra +  tejgge_r08ct05otgst +  dfgdevpiagge_r08ct05otgst +  devppimgge_r08ct05otgst +  dfgdevpiagge_r08ct06acanf +  devppimgge_r08ct06acanf +  dfgdevpiagge_r08ct06acacf +  piagge_r08ct07sedpu +  pimgge_r08ct07sedpu +  dfgpimpiagge_r08ct07sedpu +  devppimgge_r08ct07sedpu +  pimgge_r18ct05pobso +  dfgpimpiage_r18ct05pobso +  devppimge_r18ct05pobso +  dfgpimpiage_r18ct05popso +  devppimge_r18ct05popso +  devppimge_r18ct05biser +  tdvgge_r18ct05dotra +  devppimge_r18ct05dotra +  piagge_r18ct05otgst +  pimgge_r18ct05otgst +  dfgdevpiagge_r18ct05otgst +  devppimgge_r18ct05otgst +  dfgpimpiagge_r18ct06acacf +  piagge_r18ct07sedpu +  devppimgge_r18ct07sedpu +  piagkft_reod +  dfgpimpiagkft_reod +  devppimgkft_reod +  tdvgkft_redr +  dfgpimpiagkft_redr +  dfgdevpiagkft_redr +  dfgpimpiagkft_rooc +  devppimgkft_rooc +  devppimgkft_dotr +  devppimgkft_rdet +  piagkftr07_rdet +  dfgpimpiagkftr07_rdet +  dfgdevpiagkftr07_rdet +  devppimgkftr07_rdet +  dfgpimpiagkftr08_rdet +  tejgfun_f1ct05agro +  tejgfun_f1ct05amb +  tejgfun_f1ct05come +  tejgfun_f1ct05cydep +  tdvgfun_f1ct05cydep +  devppimfun_f1ct05cydep +  dfgdevpiagfun_f1ct05edu +  devppimfun_f1ct05edu +  tejgfun_f1ct05opseg +  dfgpimpiafun_f1ct05opseg +  devppimfun_f1ct05opseg +  tejgfun_f1ct05pgercon +  devppimfun_f1ct05pgercon +  dfgpimpiafun_f1ct05salud +  tejgfun_f1ct05san +  dfgdevpiagfun_f1ct05san +  devppimfun_f1ct05san +  piagfun_f1ct05trans +  tejgfun_f1ct05trans +  dfgdevpiagfun_f1ct05trans +  devppimfun_f1ct05trans +  devppimfun_f1ct05turi +  tejgfun_f1ct05viv +  dfgpimpiafun_f1ct05viv +  devppimfun_f1ct05viv +  devppimfun_f1ct06agro +  dfgdevpiagfun_f1ct06amb +  devppimfun_f1ct06amb +  dfgdevpiagfun_f1ct06come +  tdvgfun_f1ct06cydep +  devppimfun_f1ct06cydep +  piagfun_f1ct06edu +  tejgfun_f1ct06opseg +  dfgdevpiagfun_f1ct06opseg +  devppimfun_f1ct06opseg +  dfgpimpiafun_f1ct06pgercon +  devppimfun_f1ct06pgercon +  tejgfun_f1ct06protsoc +  dfgpimpiafun_f1ct06protsoc +  devppimfun_f1ct06protsoc +  dfgpimpiafun_f1ct06salud +  devppimfun_f1ct06salud +  dfgpimpiafun_f1ct06san +  devppimfun_f1ct06san +  tdvgfun_f1ct06trans +  tejgfun_f1ct06viv +  devppimfun_f1ct06viv +  piagfun_f2ct05agro +  tdvgfun_f2ct05agro +  dfgdevpiagfun_f2ct05agro +  devppimfun_f2ct05agro +  dfgpimpiafun_f2ct05amb +  dfgdevpiagfun_f2ct05amb +  devppimfun_f2ct05amb +  tejgfun_f2ct05come +  dfgpimpiafun_f2ct05come +  dfgdevpiagfun_f2ct05come +  dfgpimpiafun_f2ct05cydep +  devppimfun_f2ct05cydep +  piagfun_f2ct05edu +  pimgfun_f2ct05edu +  dfgpimpiafun_f2ct05edu +  devppimfun_f2ct05edu +  tejgfun_f2ct05energia +  devppimfun_f2ct05energia +  piagfun_f2ct05ind +  pimgfun_f2ct05ind +  devppimfun_f2ct05opseg +  piagfun_f2ct05protsoc +  devppimfun_f2ct05protsoc +  piagfun_f2ct05san +  pimgfun_f2ct05san +  tejgfun_f2ct05san +  dfgpimpiafun_f2ct05san +  devppimfun_f2ct05san +  dfgpimpiafun_f2ct05trans +  devppimfun_f2ct05trans +  dfgdevpiagfun_f2ct05turi +  tejgfun_f2ct05viv +  tdvgfun_f2ct05viv +  dfgpimpiafun_f2ct05viv +  piagfun_f2ct06agro +  dfgpimpiafun_f2ct06agro +  devppimfun_f2ct06agro +  piagfun_f2ct06amb +  tejgfun_f2ct06amb +  dfgpimpiafun_f2ct06amb +  devppimfun_f2ct06amb +  piagfun_f2ct06come +  pimgfun_f2ct06come +  tdvgfun_f2ct06come +  dfgpimpiafun_f2ct06come +  dfgdevpiagfun_f2ct06come +  devppimfun_f2ct06come +  dfgpimpiafun_f2ct06comunica +  dfgdevpiagfun_f2ct06comunica +  tejgfun_f2ct06cydep +  tdvgfun_f2ct06cydep +  dfgpimpiafun_f2ct06cydep +  devppimfun_f2ct06cydep +  devppimfun_f2ct06edu +  piagfun_f2ct06energia +  dfgpimpiafun_f2ct06energia +  dfgdevpiagfun_f2ct06energia +  tejgfun_f2ct06ind +  devppimfun_f2ct06ind +  piagfun_f2ct06opseg +  pimgfun_f2ct06opseg +  dfgdevpiagfun_f2ct06opseg +  devppimfun_f2ct06opseg +  piagfun_f2ct06pgercon +  pimgfun_f2ct06pgercon +  tejgfun_f2ct06pgercon +  dfgdevpiagfun_f2ct06pgercon +  devppimfun_f2ct06pgercon +  piagfun_f2ct06protsoc +  pimgfun_f2ct06protsoc +  tejgfun_f2ct06protsoc +  dfgdevpiagfun_f2ct06protsoc +  devppimfun_f2ct06protsoc +  piagfun_f2ct06salud +  tejgfun_f2ct06salud +  dfgpimpiafun_f2ct06salud +  devppimfun_f2ct06salud +  piagfun_f2ct06san +  tejgfun_f2ct06san +  dfgpimpiafun_f2ct06san +  dfgdevpiagfun_f2ct06san +  devppimfun_f2ct06san +  piagfun_f2ct06trans +  pimgfun_f2ct06trans +  dfgdevpiagfun_f2ct06trans +  devppimfun_f2ct06trans +  piagfun_f2ct06turi +  pimgfun_f2ct06turi +  tejgfun_f2ct06turi +  tdvgfun_f2ct06turi +  devppimfun_f2ct06turi +  tdvgfun_f2ct06viv +  devppimfun_f2ct06viv +  devppimfun_f3ct05opseg +  tejgfun_f3ct05pgercon +  tdvgfun_f3ct05pgercon +  tdvgfun_f3ct06opseg +  dfgpimpiafun_f3ct06opseg +  tejgfun_f3ct06pgercon +  devppimfun_f3ct06pgercon +  dfgpimpiafun_f4ct05agro +  devppimfun_f4ct05agro +  piagfun_f4ct05amb +  devppimfun_f4ct05amb +  dfgpimpiafun_f4ct05come +  devppimfun_f4ct05come +  tdvgfun_f4ct05cydep +  devppimfun_f4ct05cydep +  dfgpimpiafun_f4ct05opseg +  devppimfun_f4ct05opseg +  pimgfun_f4ct05pgercon +  tdvgfun_f4ct05pgercon +  tejgfun_f4ct05protsoc +  tejgfun_f4ct05salud +  dfgpimpiafun_f4ct05salud +  devppimfun_f4ct05salud +  dfgpimpiafun_f4ct05san +  tejgfun_f4ct05trans +  devppimfun_f4ct05trans +  tejgfun_f4ct05turi +  dfgpimpiafun_f4ct05turi +  devppimfun_f4ct05turi +  tdvgfun_f4ct05viv +  dfgpimpiafun_f4ct05viv +  devppimfun_f4ct05viv +  tdvgfun_f4ct06agro +  devppimfun_f4ct06come +  devppimfun_f4ct06cydep +  tejgfun_f4ct06edu +  dfgpimpiafun_f4ct06edu +  devppimfun_f4ct06edu +  piagfun_f4ct06pgercon +  dfgpimpiafun_f4ct06pgercon +  dfgdevpiagfun_f4ct06pgercon +  devppimfun_f4ct06pgercon +  tejgfun_f4ct06protsoc +  devppimfun_f4ct06protsoc +  dfgpimpiafun_f4ct06salud +  devppimfun_f4ct06salud +  dfgpimpiafun_f4ct06san +  devppimfun_f4ct06trab +  dfgpimpiafun_f4ct06turi +  devppimfun_f4ct06turi +  piagfun_f5ct05agro +  tejgfun_f5ct05agro +  dfgdevpiagfun_f5ct05amb +  devppimfun_f5ct05come +  dfgpimpiafun_f5ct05comunica +  devppimfun_f5ct05comunica +  devppimfun_f5ct05cydep +  dfgdevpiagfun_f5ct05dpub +  tdvgfun_f5ct05edu +  dfgpimpiafun_f5ct05edu +  devppimfun_f5ct05edu +  tejgfun_f5ct05energia +  tdvgfun_f5ct05energia +  dfgpimpiafun_f5ct05energia +  dfgdevpiagfun_f5ct05energia +  devppimfun_f5ct05energia +  dfgpimpiafun_f5ct05ind +  pimgfun_f5ct05opseg +  dfgpimpiafun_f5ct05opseg +  piagfun_f5ct05pgercon +  devppimfun_f5ct05pgercon +  dfgpimpiafun_f5ct05protsoc +  devppimfun_f5ct05protsoc +  piagfun_f5ct05salud +  devppimfun_f5ct05salud +  piagfun_f5ct05san +  dfgdevpiagfun_f5ct05san +  devppimfun_f5ct05san +  devppimfun_f5ct05trab +  piagfun_f5ct05turi +  pimgfun_f5ct05turi +  dfgpimpiafun_f5ct05turi +  dfgdevpiagfun_f5ct06agro +  devppimfun_f5ct06amb +  pimgfun_f5ct06come +  devppimfun_f5ct06come +  devppimfun_f5ct06comunica +  devppimfun_f5ct06cydep +  piagfun_f5ct06edu +  dfgdevpiagfun_f5ct06edu +  devppimfun_f5ct06ind +  devppimfun_f5ct06pesca +  dfgpimpiafun_f5ct06pgercon +  pimgfun_f5ct06protsoc +  pimgfun_f5ct06trab +  tejgfun_f5ct06trab +  dfgdevpiagfun_f5ct06trab +  devppimfun_f5ct06trans +  dfgdevpiagfun_f5ct06viv +  devppimfun_f5energia +  piagfun_f5r07ct05agro +  tejgfun_f5r07ct05agro +  dfgdevpiagfun_f5r07ct05agro +  devppimfun_f5r07ct05agro +  dfgdevpiagfun_f5r07ct05amb +  dfgpimpiafun_f5r07ct05come +  devppimfun_f5r07ct05come +  piagfun_f5r07ct05comunica +  dfgpimpiafun_f5r07ct05comunica +  tejgfun_f5r07ct05cydep +  tdvgfun_f5r07ct05cydep +  dfgpimpiafun_f5r07ct05cydep +  devppimfun_f5r07ct05cydep +  devppimfun_f5r07ct05dpub +  piagfun_f5r07ct05edu +  tejgfun_f5r07ct05edu +  dfgdevpiagfun_f5r07ct05edu +  devppimfun_f5r07ct05edu +  tejgfun_f5r07ct05energia +  tdvgfun_f5r07ct05energia +  dfgpimpiafun_f5r07ct05energia +  devppimfun_f5r07ct05energia +  piagfun_f5r07ct05ind +  dfgdevpiagfun_f5r07ct05ind +  devppimfun_f5r07ct05ind +  dfgpimpiafun_f5r07ct05opseg +  dfgdevpiagfun_f5r07ct05pgercon +  devppimfun_f5r07ct05pgercon +  dfgpimpiafun_f5r07ct05protsoc +  dfgdevpiagfun_f5r07ct05protsoc +  devppimfun_f5r07ct05protsoc +  tdvgfun_f5r07ct05salud +  dfgdevpiagfun_f5r07ct05salud +  piagfun_f5r07ct05san +  tejgfun_f5r07ct05san +  dfgdevpiagfun_f5r07ct05san +  devppimfun_f5r07ct05san +  piagfun_f5r07ct05trans +  tejgfun_f5r07ct05trans +  tdvgfun_f5r07ct05trans +  dfgdevpiagfun_f5r07ct05trans +  devppimfun_f5r07ct05trans +  piagfun_f5r07ct05turi +  pimgfun_f5r07ct05turi +  tejgfun_f5r07ct05turi +  dfgpimpiafun_f5r07ct05turi +  pimgfun_f5r07ct05viv +  tejgfun_f5r07ct05viv +  dfgpimpiafun_f5r07ct05viv +  dfgdevpiagfun_f5r07ct05viv +  devppimfun_f5r07ct05viv +  tejgfun_f5r07ct06agro +  tdvgfun_f5r07ct06agro +  devppimfun_f5r07ct06agro +  piagfun_f5r07ct06amb +  pimgfun_f5r07ct06amb +  tejgfun_f5r07ct06amb +  devppimfun_f5r07ct06amb +  pimgfun_f5r07ct06come +  tejgfun_f5r07ct06come +  tdvgfun_f5r07ct06come +  dfgdevpiagfun_f5r07ct06come +  devppimfun_f5r07ct06come +  devppimfun_f5r07ct06comunica +  tejgfun_f5r07ct06cydep +  devppimfun_f5r07ct06cydep +  tejgfun_f5r07ct06edu +  dfgdevpiagfun_f5r07ct06edu +  devppimfun_f5r07ct06edu +  piagfun_f5r07ct06energia +  devppimfun_f5r07ct06energia +  tdvgfun_f5r07ct06ind +  dfgpimpiafun_f5r07ct06ind +  piagfun_f5r07ct06opseg +  tejgfun_f5r07ct06opseg +  devppimfun_f5r07ct06opseg +  pimgfun_f5r07ct06pgercon +  tejgfun_f5r07ct06pgercon +  dfgpimpiafun_f5r07ct06pgercon +  devppimfun_f5r07ct06pgercon +  dfgdevpiagfun_f5r07ct06prevsoc +  tejgfun_f5r07ct06protsoc +  tdvgfun_f5r07ct06protsoc +  devppimfun_f5r07ct06protsoc +  piagfun_f5r07ct06salud +  devppimfun_f5r07ct06salud +  dfgpimpiafun_f5r07ct06san +  devppimfun_f5r07ct06trab +  dfgpimpiafun_f5r07ct06trans +  devppimfun_f5r07ct06trans +  tejgfun_f5r07ct06turi +  tdvgfun_f5r07ct06turi +  dfgdevpiagfun_f5r07ct06turi +  devppimfun_f5r07ct06turi +  devppimfun_f5r07ct06viv +  pimgfun_f5r08ct05agro +  dfgpimpiafun_f5r08ct05agro +  devppimfun_f5r08ct05agro +  devppimfun_f5r08ct05amb +  piagfun_f5r08ct05come +  devppimfun_f5r08ct05comunica +  devppimfun_f5r08ct05cydep +  piagfun_f5r08ct05edu +  tejgfun_f5r08ct05edu +  tdvgfun_f5r08ct05edu +  devppimfun_f5r08ct05edu +  piagfun_f5r08ct05energia +  tejgfun_f5r08ct05energia +  dfgpimpiafun_f5r08ct05energia +  devppimfun_f5r08ct05energia +  piagfun_f5r08ct05opseg +  devppimfun_f5r08ct05opseg +  devppimfun_f5r08ct05pgercon +  dfgpimpiafun_f5r08ct05protsoc +  devppimfun_f5r08ct05protsoc +  tejgfun_f5r08ct05salud +  piagfun_f5r08ct05san +  tejgfun_f5r08ct05san +  tdvgfun_f5r08ct05san +  dfgdevpiagfun_f5r08ct05san +  devppimfun_f5r08ct05san +  dfgpimpiafun_f5r08ct05trans +  dfgdevpiagfun_f5r08ct05trans +  devppimfun_f5r08ct05trans +  dfgpimpiafun_f5r08ct05viv +  piagfun_f5r08ct06agro +  dfgdevpiagfun_f5r08ct06agro +  devppimfun_f5r08ct06agro +  pimgfun_f5r08ct06amb +  tejgfun_f5r08ct06amb +  dfgpimpiafun_f5r08ct06amb +  devppimfun_f5r08ct06amb +  pimgfun_f5r08ct06come +  tdvgfun_f5r08ct06come +  dfgpimpiafun_f5r08ct06come +  devppimfun_f5r08ct06come +  piagfun_f5r08ct06comunica +  tdvgfun_f5r08ct06comunica +  devppimfun_f5r08ct06comunica +  piagfun_f5r08ct06cydep +  tejgfun_f5r08ct06cydep +  dfgpimpiafun_f5r08ct06cydep +  devppimfun_f5r08ct06cydep +  piagfun_f5r08ct06edu +  dfgdevpiagfun_f5r08ct06edu +  devppimfun_f5r08ct06edu +  piagfun_f5r08ct06energia +  devppimfun_f5r08ct06energia +  tdvgfun_f5r08ct06opseg +  dfgpimpiafun_f5r08ct06opseg +  devppimfun_f5r08ct06opseg +  piagfun_f5r08ct06pgercon +  tejgfun_f5r08ct06pgercon +  devppimfun_f5r08ct06pgercon +  piagfun_f5r08ct06protsoc +  tejgfun_f5r08ct06protsoc +  tdvgfun_f5r08ct06protsoc +  devppimfun_f5r08ct06protsoc +  tdvgfun_f5r08ct06salud +  dfgdevpiagfun_f5r08ct06salud +  devppimfun_f5r08ct06salud +  tejgfun_f5r08ct06san +  tdvgfun_f5r08ct06san +  devppimfun_f5r08ct06san +  dfgpimpiafun_f5r08ct06trab +  tejgfun_f5r08ct06trans +  devppimfun_f5r08ct06trans +  piagfun_f5r08ct06turi +  tejgfun_f5r08ct06turi +  devppimfun_f5r08ct06turi +  piagfun_f5r08ct06viv +  tejgfun_f5r08ct06viv +  tdvgfun_f5r08ct06viv +  devppimfun_f5r08ct06viv +  tdvgtotfun_f5r18comunica +  piagfun_f5r18ct05agro +  tdvgfun_f5r18ct05agro +  devppimfun_f5r18ct05agro +  piagfun_f5r18ct05amb +  tejgfun_f5r18ct05amb +  tdvgfun_f5r18ct05amb +  dfgpimpiafun_f5r18ct05amb +  devppimfun_f5r18ct05amb +  piagfun_f5r18ct05come +  devppimfun_f5r18ct05come +  dfgdevpiagfun_f5r18ct05comunica +  piagfun_f5r18ct05cydep +  tejgfun_f5r18ct05cydep +  dfgpimpiafun_f5r18ct05cydep +  devppimfun_f5r18ct05cydep +  pimgfun_f5r18ct05edu +  tejgfun_f5r18ct05edu +  dfgpimpiafun_f5r18ct05edu +  devppimfun_f5r18ct05edu +  piagfun_f5r18ct05energia +  devppimfun_f5r18ct05energia +  tejgfun_f5r18ct05ind +  devppimfun_f5r18ct05ind +  piagfun_f5r18ct05opseg +  tejgfun_f5r18ct05opseg +  dfgdevpiagfun_f5r18ct05opseg +  devppimfun_f5r18ct05opseg +  devppimfun_f5r18ct05pesca +  dfgdevpiagfun_f5r18ct05pgercon +  devppimfun_f5r18ct05pgercon +  tejgfun_f5r18ct05prevsoc +  pimgfun_f5r18ct05protsoc +  tejgfun_f5r18ct05protsoc +  tdvgfun_f5r18ct05protsoc +  dfgdevpiagfun_f5r18ct05protsoc +  devppimfun_f5r18ct05protsoc +  dfgdevpiagfun_f5r18ct05salud +  devppimfun_f5r18ct05salud +  piagfun_f5r18ct05san +  devppimfun_f5r18ct05san +  devppimfun_f5r18ct05trab +  piagfun_f5r18ct05trans +  tejgfun_f5r18ct05trans +  devppimfun_f5r18ct05trans +  piagfun_f5r18ct05turi +  tejgfun_f5r18ct05turi +  tdvgfun_f5r18ct05turi +  dfgpimpiafun_f5r18ct05turi +  devppimfun_f5r18ct05turi +  piagfun_f5r18ct05viv +  tejgfun_f5r18ct05viv +  dfgpimpiafun_f5r18ct05viv +  dfgdevpiagfun_f5r18ct05viv +  devppimfun_f5r18ct05viv +  devppimfun_f5r18ct06agro +  devppimfun_f5r18ct06amb +  dfgdevpiagfun_f5r18ct06come +  devppimfun_f5r18ct06come +  dfgpimpiafun_f5r18ct06ind +  devppimfun_f5r18ct06opseg +  pimgfun_f5r18ct06pesca +  piagfun_f5r18ct06pgercon +  devppimfun_f5r18ct06pgercon +  piagfun_f5r18ct06prevsoc +  devppimfun_f5r18ct06prevsoc +  devppimfun_f5r18ct06protsoc +  dfgpimpiafun_f5r18ct06salud +  devppimfun_f5r18ct06salud +  dfgpimpiafun_f5r18ct06trab +  devppimfun_f5r18ct06trab +  piagfun_f5r18ct06viv +  dfgdevpiagfun_f5r18ct06viv +  devppimfun_f5r18ct06viv +  piagtotfun_f1agro +  pimgtotfun_f1agro +  tejgtotfun_f1agro +  devppimtotfun_f1agro +  devppimtotfun_f1amb +  tejgtotfun_f1come +  dfgdevpiagtotfun_f1come +  devppimtotfun_f1come +  tejgtotfun_f1cydep +  piagtotfun_f1edu +  pimgtotfun_f1edu +  devppimtotfun_f1edu +  dfgdevpiagtotfun_f1energia +  devppimtotfun_f1energia +  piagtotfun_f1opseg +  dfgdevpiagtotfun_f1pgercon +  piagtotfun_f1prevsoc +  tejgtotfun_f1prevsoc +  tdvgtotfun_f1prevsoc +  devppimtotfun_f1prevsoc +  dfgdevpiagtotfun_f1protsoc +  piagtotfun_f1salud +  tejgtotfun_f1salud +  devppimtotfun_f1salud +  piagtotfun_f1san +  tdvgtotfun_f1san +  dfgpimpiatotfun_f1trab +  dfgpimpiatotfun_f1trans +  devppimtotfun_f1trans +  tdvgtotfun_f1turi +  devppimtotfun_f1turi +  dfgpimpiatotfun_f1viv +  piagtotfun_f2agro +  tejgtotfun_f2agro +  tdvgtotfun_f2agro +  devppimtotfun_f2agro +  piagtotfun_f2comunica +  piagtotfun_f2cydep +  pimgtotfun_f2cydep +  tejgtotfun_f2cydep +  dfgpimpiatotfun_f2cydep +  dfgpimpiatotfun_f2dpub +  devppimtotfun_f2dpub +  piagtotfun_f2edu +  pimgtotfun_f2edu +  dfgdevpiagtotfun_f2edu +  devppimtotfun_f2edu +  piagtotfun_f2energia +  devppimtotfun_f2energia +  devppimtotfun_f2ind +  dfgpimpiatotfun_f2opseg +  tejgtotfun_f2pgercon +  dfgpimpiatotfun_f2pgercon +  devppimtotfun_f2pgercon +  dfgpimpiatotfun_f2prevsoc +  devppimtotfun_f2prevsoc +  tejgtotfun_f2protsoc +  dfgdevpiagtotfun_f2protsoc +  pimgtotfun_f2salud +  dfgpimpiatotfun_f2salud +  devppimtotfun_f2salud +  piagtotfun_f2san +  tejgtotfun_f2san +  tdvgtotfun_f2san +  devppimtotfun_f2san +  dfgpimpiatotfun_f2trab +  piagtotfun_f2trans +  dfgdevpiagtotfun_f2trans +  devppimtotfun_f2trans +  piagtotfun_f2turi +  pimgtotfun_f2turi +  dfgpimpiatotfun_f2turi +  devppimtotfun_f2turi +  devppimtotfun_f2viv +  piagtotfun_f3agro +  pimgtotfun_f3agro +  tejgtotfun_f3agro +  devppimtotfun_f3agro +  dfgdevpiagtotfun_f3amb +  devppimtotfun_f3amb +  devppimtotfun_f3come +  dfgpimpiatotfun_f3comunica +  tejgtotfun_f3cydep +  tdvgtotfun_f3cydep +  devppimtotfun_f3cydep +  piagtotfun_f3edu +  tejgtotfun_f3edu +  dfgpimpiatotfun_f3edu +  dfgdevpiagtotfun_f3edu +  devppimtotfun_f3edu +  tejgtotfun_f3energia +  dfgdevpiagtotfun_f3energia +  devppimtotfun_f3energia +  tejgtotfun_f3opseg +  tdvgtotfun_f3opseg +  devppimtotfun_f3opseg +  dfgpimpiatotfun_f3pesca +  piagtotfun_f3pgercon +  devppimtotfun_f3pgercon +  tejgtotfun_f3protsoc +  dfgpimpiatotfun_f3protsoc +  devppimtotfun_f3protsoc +  dfgdevpiagtotfun_f3salud +  devppimtotfun_f3salud +  piagtotfun_f3san +  pimgtotfun_f3san +  tejgtotfun_f3san +  devppimtotfun_f3san +  devppimtotfun_f3trab +  dfgpimpiatotfun_f3trans +  devppimtotfun_f3trans +  piagtotfun_f3viv +  tejgtotfun_f3viv +  dfgdevpiagtotfun_f3viv +  devppimtotfun_f3viv +  tejgtotfun_f4agro +  devppimtotfun_f4agro +  piagtotfun_f4amb +  pimgtotfun_f4amb +  tdvgtotfun_f4amb +  devppimtotfun_f4amb +  tejgtotfun_f4cydep +  dfgdevpiagtotfun_f4cydep +  devppimtotfun_f4cydep +  tejgtotfun_f4energia +  dfgpimpiatotfun_f4energia +  dfgdevpiagtotfun_f4energia +  devppimtotfun_f4energia +  tejgtotfun_f4opseg +  devppimtotfun_f4opseg +  piagtotfun_f4pgercon +  tejgtotfun_f4pgercon +  devppimtotfun_f4pgercon +  dfgpimpiatotfun_f4prevsoc +  tejgtotfun_f4protsoc +  dfgdevpiagtotfun_f4protsoc +  devppimtotfun_f4protsoc +  piagtotfun_f4salud +  devppimtotfun_f4salud +  devppimtotfun_f4san +  dfgpimpiatotfun_f4trab +  devppimtotfun_f4trab +  devppimtotfun_f4trans +  dfgdevpiagtotfun_f4turi +  devppimtotfun_f4turi +  tdvgtotfun_f4viv +  devppimtotfun_f4viv +  piagtotfun_f5agro +  devppimtotfun_f5agro +  tejgtotfun_f5amb +  dfgpimpiatotfun_f5amb +  devppimtotfun_f5amb +  pimgtotfun_f5come +  tejgtotfun_f5come +  dfgdevpiagtotfun_f5come +  pimgtotfun_f5comunica +  piagtotfun_f5cydep +  pimgtotfun_f5cydep +  tejgtotfun_f5cydep +  dfgdevpiagtotfun_f5cydep +  devppimtotfun_f5cydep +  piagtotfun_f5dpub +  pimgtotfun_f5dpub +  tejgtotfun_f5dpub +  dfgpimpiatotfun_f5dpub +  devppimtotfun_f5dpub +  tejgtotfun_f5edu +  pimgtotfun_f5ind +  tdvgtotfun_f5ind +  devppimtotfun_f5ind +  piagtotfun_f5opseg +  pimgtotfun_f5opseg +  tejgtotfun_f5opseg +  dfgdevpiagtotfun_f5opseg +  devppimtotfun_f5opseg +  dfgpimpiatotfun_f5prevsoc +  devppimtotfun_f5prevsoc +  piagtotfun_f5protsoc +  pimgtotfun_f5protsoc +  tejgtotfun_f5protsoc +  devppimtotfun_f5protsoc +  piagtotfun_f5r07agro +  dfgdevpiagtotfun_f5r07agro +  devppimtotfun_f5r07agro +  pimgtotfun_f5r07amb +  dfgpimpiatotfun_f5r07amb +  devppimtotfun_f5r07amb +  piagtotfun_f5r07come +  tejgtotfun_f5r07come +  dfgdevpiagtotfun_f5r07come +  piagtotfun_f5r07comunica +  tejgtotfun_f5r07comunica +  dfgpimpiatotfun_f5r07comunica +  devppimtotfun_f5r07comunica +  piagtotfun_f5r07cydep +  dfgdevpiagtotfun_f5r07cydep +  devppimtotfun_f5r07cydep +  piagtotfun_f5r07dpub +  pimgtotfun_f5r07dpub +  dfgpimpiatotfun_f5r07dpub +  piagtotfun_f5r07edu +  pimgtotfun_f5r07edu +  pimgtotfun_f5r07energia +  tejgtotfun_f5r07energia +  dfgpimpiatotfun_f5r07energia +  dfgdevpiagtotfun_f5r07energia +  piagtotfun_f5r07ind +  tdvgtotfun_f5r07ind +  dfgpimpiatotfun_f5r07ind +  dfgdevpiagtotfun_f5r07ind +  piagtotfun_f5r07opseg +  pimgtotfun_f5r07opseg +  tejgtotfun_f5r07opseg +  dfgdevpiagtotfun_f5r07opseg +  devppimtotfun_f5r07opseg +  piagtotfun_f5r07pesca +  tejgtotfun_f5r07pesca +  dfgpimpiatotfun_f5r07pesca +  pimgtotfun_f5r07pgercon +  tejgtotfun_f5r07pgercon +  dfgpimpiatotfun_f5r07pgercon +  devppimtotfun_f5r07pgercon +  tejgtotfun_f5r07prevsoc +  dfgpimpiatotfun_f5r07prevsoc +  devppimtotfun_f5r07prevsoc +  piagtotfun_f5r07protsoc +  tejgtotfun_f5r07protsoc +  tdvgtotfun_f5r07protsoc +  dfgdevpiagtotfun_f5r07protsoc +  tejgtotfun_f5r07salud +  dfgdevpiagtotfun_f5r07salud +  devppimtotfun_f5r07salud +  piagtotfun_f5r07san +  pimgtotfun_f5r07san +  tejgtotfun_f5r07san +  devppimtotfun_f5r07san +  devppimtotfun_f5r07trab +  pimgtotfun_f5r07turi +  devppimtotfun_f5r07turi +  pimgtotfun_f5r07viv +  tejgtotfun_f5r07viv +  dfgpimpiatotfun_f5r07viv +  devppimtotfun_f5r07viv +  piagtotfun_f5r08agro +  tejgtotfun_f5r08agro +  dfgpimpiatotfun_f5r08agro +  tejgtotfun_f5r08amb +  dfgpimpiatotfun_f5r08amb +  tejgtotfun_f5r08come +  dfgpimpiatotfun_f5r08come +  dfgdevpiagtotfun_f5r08cydep +  pimgtotfun_f5r08edu +  dfgpimpiatotfun_f5r08edu +  devppimtotfun_f5r08edu +  piagtotfun_f5r08energia +  tejgtotfun_f5r08energia +  devppimtotfun_f5r08energia +  dfgdevpiagtotfun_f5r08ind +  tejgtotfun_f5r08opseg +  dfgpimpiatotfun_f5r08opseg +  pimgtotfun_f5r08pgercon +  dfgpimpiatotfun_f5r08pgercon +  dfgdevpiagtotfun_f5r08prevsoc +  devppimtotfun_f5r08prevsoc +  pimgtotfun_f5r08protsoc +  tejgtotfun_f5r08protsoc +  dfgpimpiatotfun_f5r08protsoc +  piagtotfun_f5r08salud +  pimgtotfun_f5r08salud +  devppimtotfun_f5r08salud +  dfgpimpiatotfun_f5r08san +  dfgdevpiagtotfun_f5r08san +  tejgtotfun_f5r08trab +  dfgdevpiagtotfun_f5r08trab +  devppimtotfun_f5r08trab +  pimgtotfun_f5r08trans +  dfgdevpiagtotfun_f5r08trans +  devppimtotfun_f5r08turi +  dfgpimpiatotfun_f5r08viv +  devppimtotfun_f5r08viv +  pimgtotfun_f5r18agro +  devppimtotfun_f5r18agro +  tejgtotfun_f5r18amb +  dfgdevpiagtotfun_f5r18amb +  devppimtotfun_f5r18amb +  piagtotfun_f5r18come +  dfgpimpiatotfun_f5r18come +  devppimtotfun_f5r18come +  piagtotfun_f5r18cydep +  dfgpimpiatotfun_f5r18cydep +  devppimtotfun_f5r18cydep +  tejgtotfun_f5r18dpub +  dfgdevpiagtotfun_f5r18dpub +  devppimtotfun_f5r18edu +  pimgtotfun_f5r18energia +  dfgdevpiagtotfun_f5r18energia +  piagtotfun_f5r18ind +  devppimtotfun_f5r18ind +  dfgpimpiatotfun_f5r18just +  dfgdevpiagtotfun_f5r18just +  piagtotfun_f5r18opseg +  tejgtotfun_f5r18opseg +  dfgdevpiagtotfun_f5r18opseg +  devppimtotfun_f5r18opseg +  piagtotfun_f5r18pesca +  devppimtotfun_f5r18pgercon +  dfgdevpiagtotfun_f5r18prevsoc +  devppimtotfun_f5r18prevsoc +  tdvgtotfun_f5r18protsoc +  devppimtotfun_f5r18protsoc +  dfgpimpiatotfun_f5r18salud +  devppimtotfun_f5r18salud +  devppimtotfun_f5r18san +  tejgtotfun_f5r18trab +  devppimtotfun_f5r18trab +  dfgpimpiatotfun_f5r18trans +  dfgdevpiagtotfun_f5r18trans +  devppimtotfun_f5r18trans +  tejgtotfun_f5r18turi +  devppimtotfun_f5r18turi +  pimgtotfun_f5r18viv +  tejgtotfun_f5r18viv +  dfgpimpiatotfun_f5r18viv +  devppimtotfun_f5r18viv +  piagtotfun_f5salud +  tdvgtotfun_f5salud +  dfgdevpiagtotfun_f5salud +  devppimtotfun_f5salud +  dfgpimpiatotfun_f5san +  piagtotfun_f5trab +  pimgtotfun_f5trab +  devppimtotfun_f5trab +  tejgtotfun_f5trans +  tdvgtotfun_f5turi +  devppimtotfun_f5turi +  piagtotfun_f5viv +  tejgtotfun_f5viv +  tdvgtotfun_f5viv +  devppimtotfun_f5viv"
y_basic_train, x_basic_train = patsy.dmatrices( formula_basic, data_train, return_type = 'dataframe' )
y_basic_test, x_basic_test = patsy.dmatrices( formula_basic, data_test, return_type = 'dataframe' )
p_basic = x_basic_train.shape[ 1 ]

Wall time: 2.76 s


#### 2.2. Generar variables dependientes

In [12]:
Y_train = data_train[ 'per_corrup2' ]
Y_test = data_test[ 'per_corrup2' ]

In [13]:
p_basic

1034

## 3. OLS

In [14]:
x_basic_train.head()

,Intercept,tejgfun_ct05pgercon,dfgpimpiafun_ct05pgercon,devppimfun_ct05pgercon,dfgpimpiafun_ct05opseg,devppimfun_ct05opseg,devppimfun_ct05trab,piagfun_ct05come,dfgdevpiagfun_ct05turi,devppimfun_ct05turi,...,piagtotfun_f5trab,pimgtotfun_f5trab,devppimtotfun_f5trab,tejgtotfun_f5trans,tdvgtotfun_f5turi,devppimtotfun_f5turi,piagtotfun_f5viv,tejgtotfun_f5viv,tdvgtotfun_f5viv,devppimtotfun_f5viv
339,1.0,15.386930,1.351333,4.289474,0.482115,4.187827,0.0,0.0,0.00,0.000000,...,0.0,0.0,0.0,14.711309,0.000000,0.000000,0.000000,12.027408,12.027408,4.453322
549,1.0,13.155292,0.146427,4.511438,0.000000,0.000000,0.0,0.0,0.03,4.615121,...,0.0,0.0,0.0,10.787709,10.308986,4.615121,0.000000,0.000000,0.000000,0.000000
255,1.0,0.000000,0.188744,4.526656,0.000000,0.000000,0.0,0.0,0.00,0.000000,...,0.0,0.0,0.0,0.000000,10.565009,4.467280,12.611541,0.000000,11.725138,3.921102
83,1.0,0.000000,2.125175,4.291729,-0.005700,4.070790,0.0,0.0,0.00,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
673,1.0,0.000000,-1.846324,4.425671,0.702300,4.379860,0.0,0.0,0.00,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,13.844683,4.553912


#### 3.1. Ajustar el modelo

In [15]:
fit_lm_basic = smf.ols( formula_basic, data = data_train ).fit()

#### 3.2. Calcular out-of-sample MSE y Standard Error

In [16]:
# Only MSE

yhat_lm_basic = fit_lm_basic.predict( data_test )
print("The mean squared error (MSE) using the basic model is equal to:", ( ( Y_test - yhat_lm_basic )**2 ).mean() )

The mean squared error (MSE) using the basic model is equal to: 3.440051490543533


In [17]:
# MSE and SE

resid_basic  = ( Y_test - yhat_lm_basic )**2

MSE_lm_basic = sm.OLS( resid_basic , np.ones( resid_basic.shape[ 0 ] ) ).fit().summary2().tables[ 1 ].iloc[ 0, 0:2 ]
MSE_lm_basic

Coef.       3.440051
Std.Err.    0.432306
Name: const, dtype: float64

#### 3.3. Calcular R Cuadrado out-of-sample

In [18]:
R2_lm_basic = 1 - ( MSE_lm_basic[ 0 ]/Y_test.var() )
print( f"The R^2 using the basic model is equal to: { R2_lm_basic }" )

The R^2 using the basic model is equal to: -3.9582318645576553


## 4. Lasso, Ridge and Elastic Net

#### 4.1. Theoretical Lasso from hdm package

In [19]:
import hdmpy as hdm

In [20]:
fit_rlasso = hdm.rlasso( x_basic_train.to_numpy() , Y_train.to_numpy().reshape( Y_train.size , 1 ) , post = False )
fit_rlasso_post = hdm.rlasso( x_basic_train.to_numpy() , Y_train.to_numpy().reshape( Y_train.size , 1 ) , post = True )

In [21]:
# Getting mean of each variable
meanx = x_basic_test.mean( axis = 0 ).values.\
                        reshape( x_basic_test.shape[ 1 ] , 1 )

# Reducing the mean
new_x1 = x_basic_test.to_numpy() - \
                    (np.ones( ( x_basic_test.shape[ 0 ] , 1 ) ) @ meanx.T)

# Getting the significant variables
x1_est_rlasso = new_x1[ :, fit_rlasso.est['index'].iloc[:, 0].to_list()]

# Getting the coef. from significant variables
beta_rlasso = fit_rlasso.est['beta'].loc[ fit_rlasso.est['index'].\
                                     iloc[:, 0].to_list(), ].to_numpy()

# yhat
yhat_rlasso = (x1_est_rlasso @ beta_rlasso) + np.mean( Y_test.to_numpy() )
residuals_rlasso = Y_test.to_numpy().reshape( Y_test.to_numpy().size, 1)  - yhat_rlasso

In [22]:
# Getting mean of each variable
meanx = x_basic_test.mean( axis = 0 ).values.\
                        reshape(x_basic_test.shape[ 1 ] , 1 )

# Reducing the mean
new_x1 = x_basic_test.to_numpy() - \
                    (np.ones( (x_basic_test.shape[ 0 ] , 1 ) ) @ meanx.T)

# Getting the significant variables
x1_est_rlasso_post = new_x1[ :, fit_rlasso_post.est['index'].iloc[:, 0].to_list()]

# Getting the coef. from significant variables
beta_rlasso_post = fit_rlasso_post.est['beta'].loc[ fit_rlasso_post.est['index'].\
                                     iloc[:, 0].to_list(), ].to_numpy()

# yhat
yhat_rlasso_post = (x1_est_rlasso_post @ beta_rlasso_post) + np.mean( Y_test.to_numpy() )
residuals_rlasso_post = Y_test.to_numpy().reshape( Y_test.to_numpy().size, 1)  - yhat_rlasso_post

In [23]:
MSE_lasso = sm.OLS( ( residuals_rlasso )**2 , np.ones( yhat_rlasso.size )  ).fit().summary2().tables[1].round(3)
MSE_lasso_post = sm.OLS( ( residuals_rlasso_post )**2  , np.ones( yhat_rlasso_post.size )  ).fit().summary2().tables[1].round(3)

R2_lasso = 1 - MSE_lasso.iloc[0, 0]/ np.var( Y_test )
R2_lasso_post = 1 - MSE_lasso_post.iloc[0, 0]/ np.var( Y_test )

print( f"The R^2 using the basic model is equal to {R2_lasso},for lasso and {R2_lasso_post} for post-lasso") # R^2 lasso/post-lasso (basic model) 

The R^2 using the basic model is equal to 0.043258472718139984,for lasso and -0.007477820395291879 for post-lasso


#### 4.2. Cross Validated Lasso, Ridge and Elastic Net

In [24]:
from sklearn.linear_model import LassoCV, RidgeCV, ElasticNetCV

In [25]:
fit_lasso_cv = LassoCV( cv = 10, fit_intercept = True, normalize = False, random_state = 0 ).fit( x_basic_train, Y_train )
fit_ridge = RidgeCV( cv = 10, fit_intercept = True, normalize = False, scoring = None ).fit( x_basic_train, Y_train )
fit_elnet = ElasticNetCV( cv = 10, fit_intercept = True, normalize = False, random_state = 0 ).fit( x_basic_train, Y_train )

yhat_lasso_cv = fit_lasso_cv.predict( x_basic_test )
yhat_ridge = fit_ridge.predict( x_basic_test )
yhat_elnet = fit_elnet.predict( x_basic_test )

residual_lasso = ( yhat_lasso_cv - Y_test )**2
residual_ridge = ( yhat_ridge - Y_test )**2
residual_elnet = ( yhat_elnet - Y_test )**2

MSE_lasso_cv = sm.OLS( residual_lasso, np.ones( Y_test.size )).fit().summary2().tables[ 1 ].round( 3 )
MSE_ridge = sm.OLS( residual_ridge, np.ones( Y_test.size )).fit().summary2().tables[ 1 ].round( 3 )
MSE_elnet = sm.OLS( residual_elnet, np.ones( Y_test.size )).fit().summary2().tables[ 1 ].round( 3 )

R2_lasso_cv = 1 - MSE_lasso_cv.iloc[ 0, 0 ] / np.var( Y_test )
R2_ridge = 1 - MSE_ridge.iloc[ 0, 0 ]  / np.var( Y_test )
R2_elnet = 1 - MSE_elnet.iloc[ 0, 0 ]  / np.var( Y_test )

print( "R^2 using cross-validation for lasso, ridge, and elastic net in the basic model: {:.5f}, {:.5f}, {:.5f}".format( R2_lasso_cv, R2_ridge, R2_elnet ) )

R^2 using cross-validation for lasso, ridge, and elastic net in the basic model: 0.04471, -3.02556, 0.04471


## 5. Non Linear Models

#### 5.1. Regression Trees

In [26]:
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree

In [27]:
# Fit the model

fit_trees = DecisionTreeRegressor( random_state = 0, min_impurity_decrease = 0.001 )
fit_trees.fit( x_basic_train, y_basic_train )

DecisionTreeRegressor(min_impurity_decrease=0.001, random_state=0)

In [28]:
# Plot the tree

# from sklearn.tree import plot_tree

# plt.figure( figsize=( 30, 20 ) )
# plot_tree( fit_trees, filled = True )
# plt.show()

In [29]:
# Determine the optimar complexity of the regression tree

s = pd.DataFrame( fit_trees.cost_complexity_pruning_path( y_basic_train, x_basic_train ) )
s.head()

,ccp_alphas,impurities
0,0.000000,7.637679
1,0.008808,7.646486
2,0.009444,7.655930
3,0.012345,7.668276
4,0.012770,7.681045


In [30]:
### PREGUNTA: DE DÓNDE SALE ESE VALOR DE ALPHA

In [31]:
# Prune the tree

fit_prunnedtree = DecisionTreeRegressor( ccp_alpha = 0.00188444410871555 )
fit_prunnedtree.fit( x_basic_train, y_basic_train )

DecisionTreeRegressor(ccp_alpha=0.00188444410871555)

In [32]:
# Plot the prunned tree

# plot_tree(fit_prunnedtree, filled=True)
# plt.show()

In [33]:
# Calculate MSE and R2 for prunned tree

y_hat_pt = fit_prunnedtree.predict( x_basic_test )
residual_pt = ( y_hat_pt - Y_test )**2
MSE_pt = sm.OLS( residual_pt, np.ones( y_hat_pt.size )).fit().summary2().tables[ 1 ].round( 3 )
R2_pt = 1 - MSE_pt.iloc[ 0, 0 ]/np.var( Y_test )
print( f"R^2 of the pruned tree: { R2_pt }" )

R^2 of the pruned tree: -0.8714444116983049


#### 5.2. Random Forest and Booested Trees

In [34]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor 

In [35]:
# random forest
fit_rf = RandomForestRegressor( n_estimators = 2000, min_samples_leaf = 5).fit( x_basic_train, Y_train )

# boosting
fit_boost = GradientBoostingRegressor( loss = 'ls', learning_rate = 0.01, n_estimators = 1000, max_depth = 2, subsample = 0.5).fit( x_basic_train, Y_train )

# Evaluating the methods
yhat_rf = fit_rf.predict( x_basic_test )
residual_rf = ( yhat_rf - Y_test )**2
yhat_boost = fit_boost.predict( x_basic_test )
residual_bst = ( yhat_boost - Y_test )**2

# Calculate MSE
MSE_rf = sm.OLS( residual_pt, np.ones( yhat_rf.size )).fit().summary2().tables[ 1 ].round( 3 )
MSE_bst = sm.OLS( residual_bst, np.ones( yhat_boost.size )).fit().summary2().tables[ 1 ].round( 3 )

# Calculate R2
R2_rf = 1 - MSE_rf.iloc[ 0, 0 ] / Y_test.var()
R2_boost = 1 - MSE_bst.iloc[ 0, 0 ] / Y_test.var()

In [36]:
print("R^2 of the random forest and boosted trees:{:.5f}, {:.5f}".format( R2_rf, R2_boost ))

R^2 of the random forest and boosted trees:-0.86075, 0.04728


## 6. Resultados

In [37]:
table = pd.DataFrame(columns=["MSE", "S.E for MSE", "R-squared"]) 
table.loc[0]  = [MSE_lm_basic[0], MSE_lm_basic[1], R2_lm_basic]
table.loc[1]  = [MSE_lasso.iloc[0, 0], MSE_lasso.iloc[0, 1], R2_lasso]
table.loc[2]  = [MSE_lasso.iloc[0, 0], MSE_lasso_post.iloc[0, 1], R2_lasso_post]
table.loc[3]  = [MSE_lasso_cv.iloc[0, 0], MSE_lasso_cv.iloc[0, 1], R2_lasso_cv]
table.loc[4]  = [MSE_ridge.iloc[0, 0], MSE_ridge.iloc[0, 1], R2_ridge]
table.loc[5]  = [MSE_elnet.iloc[0, 0], MSE_elnet.iloc[0, 1], R2_elnet]
table.loc[6] = [MSE_rf.iloc[0, 0], MSE_rf.iloc[0, 1], R2_rf]
table.loc[7] = [MSE_bst.iloc[0, 0], MSE_bst.iloc[0, 1], R2_boost]
table.loc[8] = [MSE_pt.iloc[0, 0], MSE_pt.iloc[0, 1], R2_pt]
models_row = [ "Least Squares (basic)", "Lasso", "Post_Lasso", "Cross-Validated lasso", 
               "Cross-Validated ridge", "Cross-Validated elnet", "Random Forest", 
               "Boosted Trees", "Pruned Tree" ]
table.insert( 0, "Models", models_row )
table

,Models,MSE,S.E for MSE,R-squared
0,Least Squares (basic),3.440051,0.432306,-3.958232
1,Lasso,0.660000,0.078000,0.043258
2,Post_Lasso,0.660000,0.081000,-0.007478
3,Cross-Validated lasso,0.659000,0.077000,0.044708
4,Cross-Validated ridge,2.777000,0.338000,-3.025562
5,Cross-Validated elnet,0.659000,0.077000,0.044708
6,Random Forest,1.291000,0.138000,-0.860750
7,Boosted Trees,0.661000,0.076000,0.047284
8,Pruned Tree,1.291000,0.138000,-0.871444
